In [ ]:
"""Processing yara rules in Python."""

# Обработка yara-правил на языке Python

[YARA](https://virustotal.github.io/yara/) является мощным средством идентификации и классификации вредоносного ПО. Исследователи вредоносных программ могут создавать правила ```YARA``` на основе текстовой или двоичной информации, содержащейся в образце. Эти правила состоят из набора строк и логического выражения, которое определяет его логику. Как только правило написано, вы можете использовать его для сканирования файлов с применением утилиты ```YARA``` или использовать модуль [yara-python](https://github.com/VirusTotal/yara-python) для интеграции с вашими инструментальными средствами.

Подробнее о написании правил YARA можно узнать на [странице](https://yara.readthedocs.io/en/v4.2.3/writingrules.html).

Полезные ссылки по генерации правил:
- [How to Write Simple but Sound Yara Rules](https://www.nextron-systems.com/2015/02/16/write-simple-sound-yara-rules/)
- [yarGen](https://github.com/Neo23x0/yarGen)

Исходные файлы для блокнота находятся по [ссылке](https://github.com/dm-fedorov/infosec/tree/master/re-tools/yara-rules) и по [ссылке](https://github.com/dm-fedorov/infosec/tree/master/re-tools/samples).

Скачиваем архив с правилами для работы в Colab:

In [ ]:
!wget https://dfedorov.spb.ru/infosec/yara/yara-rules.zip

In [ ]:
!unzip yara-rules.zip

Скачиваем архив с файлами для исследования:

In [ ]:
!wget https://dfedorov.spb.ru/infosec/re/samples.zip

In [ ]:
!unzip samples.zip

In [ ]:
!apt-get install yara

In [ ]:
!pip3 install yara-python

## Основы правил YARA

После установки следующим шагом будет создание правил ```YARA```; эти правила могут быть общими или очень конкретными и могут быть созданы с помощью любого текстового редактора.

Рассмотрим в качестве примера простое правило ```YARA```, которое ищет подозрительные строки в любом файле, а именно:

```
rule suspicious_strings
{
strings:
  $a = "Synflooding"
  $b = "Portscanner"
  $c = "Keylogger"
condition:
  ($a or $b or $c)
}
```

Правило ```YARA``` состоит из следующих компонентов:
- *идентификатор правила*: это имя, которое описывает правило (```suspicious_strings``` в предыдущем примере). Идентификаторы правила могут содержать любой буквенно-цифровой символ и знак подчеркивания, но первый символ не может быть цифрой. Идентификаторы правила чувствительны к регистру, и их количество не может превышать 128 символов;
- *определение строки*: это раздел, где определены строки (текст, шестнадцатеричные или регулярные выражения), которые будут частью правила. Эта секция может быть опущена, если правило не опирается на какие-либо строки. Каждая строка имеет идентификатор, состоящий из символа ```$```, за которым следует последовательность буквенно-цифровых символов и подчеркивания. Исходя из предыдущего правила, рассматривайте ```$a```, ```$b``` и ```$c``` как переменные, содержащие значения. Эти переменные затем используются в секции условий;
- *секция условий*: это не дополнительная секция. Здесь находится логика правила. Эта секция должна содержать логическое выражение, указывающее условие, при котором правило будет соответствовать или нет.

Следующим шагом будет использование утилиты ```yara``` для сканирования файлов. В предыдущем примере правило искало три подозрительные строки (определенные в ```$a```, ```$b``` и ```$c```) и было основано на условии. Правило соответствовало, если какая-либо из трех строк присутствовала в файле.

Правило было сохранено как ```suspicious_01.yara```:

In [ ]:
!ls

In [ ]:
!yara -r yara-rules/suspicious_01.yara samples

Предыдущее правило по умолчанию будет соответствовать ASCII-строкам и выполнять сравнение с учетом регистра символов. Если вы хотите, чтобы правило обнаруживало как ASCII-, так и Юникод-строки, укажите модификатор ```ascii``` и ```wide``` рядом со строкой. Модификатор ```nocase``` выполнит сравнение с без учета регистра символов (например, Synflooding, synflooding, sYnflooding и т. д.).

Модифицированное правило для реализации данного сравнения и поиска ASCII- и Unicode-строк показано ниже:

```
rule suspicious_strings
{
strings:
  $a = "Synflooding" ascii wide nocase
  $b = "Portscanner" ascii wide nocase
  $c = "Keylogger" ascii wide nocase
condition:
  ($a or $b or $c)
}
```
При выполнении предыдущего правила был идентифицирован документ (```v_01.txt```), содержащий Юникод-строки:

In [ ]:
!yara -r yara-rules/suspicious_02.yara samples

Если вы собираетесь искать строки в исполняемом файле, то можете создать правило, как показано ниже.

```$mz at 0``` в условии указывает ```YARA``` искать сигнатуру ```4D 5A``` (первые два байта PE-файла) в начале файла; это гарантирует, что сигнатура срабатывает только для исполняемых файлов ```PE```. Текстовые строки заключены в двойные кавычки, тогда как шестнадцатеричные строки заключены в фигурные скобки, как в переменной ```$mz```:

```
rule suspicious_strings
{
strings:
  $mz = {4D 5A}
condition:
  ($mz at 0)
}
```

Теперь при выполнении предыдущего правила обнаружены только исполняемые PE-файлы:

In [ ]:
!yara -r yara-rules/suspicious_03.yara samples

Следующее правило ```YARA``` обнаруживает исполняемый PE файл, содержащий встроенный документ Microsoft Office. Правило сработает, если будет найдена шестнадцатеричная строка со смещением больше 1024 байтов (PE-заголовок пропускается), а ```filesize``` определяет конец файла:

```
rule embedded_office_document
{
meta:
  description = "Detects embedded office document"
strings:
  $mz = {4D 5A}
  $a = {D0 CF 11 E0 A1 B1 1A E1}
condition:
  ($mz at 0) and $a in (1024..filesize)
}
```

```YARA``` может использоваться для обнаружения шаблонов в любом файле.

Следующее правило обнаруживает связь различных вариантов вредоносной программы `Gh0stRAT` (см. [тут](https://attack.mitre.org/software/S0032/)) в наборах сетевого трафика (pcap формат):

```
rule Gh0stRat_communications
{
meta:
  Description = "Detects the Gh0stRat communication in Packet Captures"
strings:
  $gst1 = {47 68 30 73 74 ?? ?? 00 00 ?? ?? 00 00 78 9c}
  $gst2 = {63 62 31 73 74 ?? ?? 00 00 ?? ?? 00 00 78 9c}
  $gst3 = {30 30 30 30 30 30 30 30 ?? ?? 00 00 ?? ?? 00 00 78 9c}
  $gst4 = {45 79 65 73 32 ?? ?? 00 00 ?? ?? 00 00 78 9c}
  $gst5 = {48 45 41 52 54 ?? ?? 00 00 ?? ?? 00 00 78 9c}
  $any_variant = /.{5,16}\x00\x00..\x00\x00\x78\x9c/
condition:
  any of ($gst*) or ($any_variant)
}
```